In [1]:
import numpy as np
from scipy.stats import chi2
from random import random, gauss, gammavariate
from tabulate import tabulate
from math import exp, comb, log, sqrt, erf

## Funciones generales
def generar_muestra(generador, n):
    """
    Genera una muestra de tamaño n de una variable aleatoria
    Args:
        generador:    Generador de la variable aleatoria
        n        :    Tamaño de la muestra
    """
    datos = []
    for _ in range(n):
        datos.append(generador())
    return datos


def generar_binomial(N, P):
    """
    Genera un valor de la variable aleatoria binomial usando el metodo de la 
    transformada inversa
    Args:
        N: Cantidad de ensayos independientes
        P: Probabilidad de exito de ensayo
    """
    prob = (1 - P) ** N
    c = P / (1 - P)
    F = prob
    i = 0
    U = random()
    while U >= F:
        prob *= c * (N-i) / (i+1)
        F += prob
        i += 1
    return i


def generar_exp(Lambda):
    """
    Genera una variable aleatoria exponencial usando el metodo de la transformada
    inversa
    Args:
        Lambda: Parametro de la variable aleatoria exponencial
    """
    U = random()
    return -log(U) / Lambda


def uniforme_d(n, m):
    """
    Genera numeros aleatorios entre n y m (inclusive) 
    """
    U = int(random() * (m - n + 1))  # 0 <= U < (m-n+1) Equivale a 0 <= U <= (m-n)
    U = U + n                        # Equivale a n <= U <= m
    return U


def permutar(a):
    """
    Permuta el arreglo a
    args: 
        a: Arreglo a permutar
    return:
        a_p: Arreglo permutado
    """
    N = len(a)
    a_p = a.copy()
    for i in range(N):
        U = uniforme_d(i, N-1)             # Numero aleatorio entre [i, N-1]
        a_p[U], a_p[i] = a_p[i], a_p[U]    # Permutamos un elemento del arreglo
    return a_p

## Funciones generales (Test para datos discretos)
def calcular_T(N, p):
    """
    Calcula el estadistico T dado en el test chi-cuadrado de 
    Pearson.
    ¡Ambas entradas deben estar ordenadas de menor a mayor respecto
    al valor de variable aleatoria al que hacen referencia!
    Args:
        N:    Frecuencia observada de la muestra
        p:    Probabilidad de los valores posibles de la variable aleatoria
    """
    n = sum(N)    # Tamaño de la muestra
    T = sum((N - n*p)**2 / (n*p))
    return T


def p_valor_pearson(t, grados_libertad):
    """
    Calcula el p-valor usando los resultados teoricos que dictan:
    Ph0(T >= t) = P(X_k-1 >= t)
    Args:
        t              :    Resultado del estadistico en la muestra
        grados_libertad:    Grados de libertad de la variable chi-cuadrada
    """
    acumulada_chi = chi2.cdf(t, grados_libertad)
    p_valor = 1 - acumulada_chi
    return p_valor


def generar_frecuencia_observada(n, p_masa, k):
    """
    Genera la frecuencia observada usando una variable aleatoria binomial
    Args:
        n     :   Tamaño de la muestra
        p_masa:   Probabilidad masa de los valores posibles de la distribucion 
                  de la hipotesis nula
        k     :   Cantidad total de valores posibles de la variable con distribucion
                  dada en la hipotesis nula
    """
    N = np.zeros(k)
    N[0] = generar_binomial(n, p_masa[0])
    for i in range(1, k-1):
        N[i] = generar_binomial(
            n - sum(N[:i]), p_masa[i] / (1 - sum(p_masa[:i])))
    N[k-1] = n - sum(N[:k-1])
    return np.array(N)


def estimar_p_valor_pearson(t, Nsim, p, k, n):
    """
    Estima el p_valor por medio de una simulacion P(T >= t)
    Args:
        t   :   Valor del estadistico en la muestra original
        Nsim:   Cantidad de simulaciones a realizar
        p   :   Probabilidad masa de los valores posibles de la distribucion 
                de la hipotesis nula
        k   :   Cantidad total de valores posibles de la variable con distribucion
                dada en la hipotesis nula
        n   :   Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        # Frecuencia observada en la simulacion
        N_sim = generar_frecuencia_observada(n, p, k)
        # Estadistico de la simulacion
        t_sim = calcular_T(N_sim, p)
        if t_sim >= t:
            p_valor += 1
    return p_valor / Nsim

## Funciones generales (Test para datos continuos)
def calcular_D(muestra, acumulada_h):
    """
    Calcula el estadistico dado en el test de Kolmogorov-Smirnov
    Args:
        muestra    :    Muestra original de datos
        acumulada_h:    Acumulada de la distribucion de la hipotesis nula
    """
    n = len(muestra)
    m_copy = list.copy(muestra)
    m_copy.sort()
    m_copy = np.array(m_copy)

    # Vectores_auxiliares
    F = np.array(list(map(acumulada_h, m_copy)))
    j = np.array(list(range(1, n+1))) / n
    j_1 = j - 1/n

    m1 = max(j - F)
    m2 = max(F - j_1)
    D = max(m1, m2)
    return D


def acumulada_uniforme(x):
    """
    Acumulada de la distribucion uniforme (U(0, 1))
    """
    if x < 0:
        return 0
    elif x < 1:
        return x
    else:
        return 1


def estimar_p_valor_k(d, Nsim, n):
    """
    Estima el p-valor por medio de simulacion P(D >= d)
    args:
        d   :    Valor del estadistico en la muestra original
        Nsim:    Cantidad de simulaciones a realizar
        n   :    Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        muestra_sim = generar_muestra(random, n)    # Muestra de uniformes
        muestra_sim.sort()
        d_sim = calcular_D(muestra_sim, acumulada_uniforme)
        if d_sim >= d:
            p_valor += 1
    return p_valor / Nsim

In [14]:
## Ejercicio 7

N = np.array([141, 291, 132])
p = np.array([1/4, 1/2, 1/4])
n = 564
t = calcular_T(N, p)
k = 3

# Prueba de pearson
p_valor = p_valor_pearson(t, grados_libertad=k - 1)

# Simulacion
Nsim = 10**3
p_valor_est = estimar_p_valor_pearson(t, Nsim, p, k, n)

data = [[p_valor, p_valor_est]]
headers = ["P-valor Pearson", "P-valor simulacion"]
print(tabulate(data, headers=headers, tablefmt="pretty"))

+--------------------+--------------------+
|  P-valor Pearson   | P-valor simulacion |
+--------------------+--------------------+
| 0.6499557054800363 |       0.642        |
+--------------------+--------------------+


In [15]:
## Ejercicio 8

N = np.array([158, 172, 164, 181, 160, 165])
p = np.array([1/6] * 6)
n = 1000
t = calcular_T(N, p)
k = 6

# Prueba de pearson
p_valor = p_valor_pearson(t, grados_libertad=k - 1)

# Simulacion
Nsim = 10**3
p_valor_est = estimar_p_valor_pearson(t, Nsim, p, k, n)

data = [[p_valor, p_valor_est]]
headers = ["P-valor Pearson", "P-valor simulacion"]
print(tabulate(data, headers=headers, tablefmt="pretty"))

+--------------------+--------------------+
|  P-valor Pearson   | P-valor simulacion |
+--------------------+--------------------+
| 0.8237195392577814 |       0.829        |
+--------------------+--------------------+


In [16]:
## Ejercicio 9

muestra = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]

d = calcular_D(muestra, acumulada_uniforme)
n = len(muestra)

# Simulacion
Nsim = 10**3
p_valor = estimar_p_valor_k(d, Nsim, n)

data = [[p_valor, d]]
headers = ["p-valor estimado", "Estadistico"]

print(tabulate(data, headers=headers, tablefmt="pretty"))

+------------------+-------------+
| p-valor estimado | Estadistico |
+------------------+-------------+
|       0.55       |    0.24     |
+------------------+-------------+


In [18]:
## Ejercicio 10
def expo(x):
    if x < 0:
        return 0
    else:
        return 1 - exp(-(1/50) * x)

muestra = [86, 133, 75, 22, 11, 144, 78, 122, 8, 146, 33, 41, 99]

d = calcular_D(muestra, expo)
n = len(muestra)

# Simulacion
Nsim = 10**4
p_valor = estimar_p_valor_k(d, Nsim, n)

data = [[p_valor, d]]
headers = ["p-valor estimado", "Estadistico"]

print(tabulate(data, headers=headers, tablefmt="pretty"))

+------------------+--------------------+
| p-valor estimado |    Estadistico     |
+------------------+--------------------+
|      0.0288      | 0.3922544552361856 |
+------------------+--------------------+


In [3]:
## Ejercicio 11
def masa(x, p):
    N = 8
    if x < 0:
        return 0
    else:
        return comb(N, x) * p**x * (1-p)**(N-x)

muestra = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
n = len(muestra)
N = [0, 1, 2, 4, 1, 1, 2, 5, 2]
k = 9

p_est = sum(muestra) / n
p_est = p_est / 8

p = np.array(list(map(lambda x: masa(x, p_est), list(range(0, 9)))))      
t = calcular_T(N, p)

p_valor = p_valor_pearson(t, grados_libertad = k - 2)   # k - 1 - m donde m es el numero de parametros estimados


# Simulacion
Nsim = 10**4
p_valor_sim = 0
for _ in range(Nsim):
    # Muestra generada a partir de la distribucion con el parametro estimado
    # en la muestra original
    muestra_sim = generar_muestra(lambda: generar_binomial(8, p_est), n)
    
    # Re-estimamos el parametro p pero ahora con esta muestra
    p_sim_est = sum(muestra_sim) / len(muestra_sim) / 8
    
    # Calculamos la frecuencia observada en la muestra
    N_sim = np.zeros(9)
    for dato in muestra_sim:
        N_sim[dato] += 1
        
    # Calculamos la probabilidad de masa teorica de cada valor posible con el parametro
    # estimado con la muestra de la simulacion
    p_sim = np.array(list(map(lambda x: masa(x, p_sim_est), list(range(0, 9)))))
    # Calculamos el valor del estadistico de la simulacion
    t_sim = calcular_T(N_sim, p_sim)

    if t_sim > t:
        p_valor_sim += 1
        
p_valor_sim = p_valor_sim / Nsim

datos   = [[t, p_valor, p_valor_sim]]
headers = ["Estadistico T", "P-valor por Pearson", "P-valor simulado"]

print(tabulate(datos, headers=headers, tablefmt="pretty"))

+--------------------+------------------------+------------------+
|   Estadistico T    |  P-valor por Pearson   | P-valor simulado |
+--------------------+------------------------+------------------+
| 31.499330934155324 | 5.0279943204278865e-05 |      0.0103      |
+--------------------+------------------------+------------------+


In [4]:
## Ejercicio 12
p = np.array([0.31, 0.22, 0.12, 0.10, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01])
N = np.array([188, 138, 87, 65, 48, 32, 30, 34, 13, 2])
n = sum(N)
t = calcular_T(N, p)
k = 10

# Prueba de pearson
p_valor = p_valor_pearson(t, grados_libertad=k - 1)

# Simulacion
Nsim = 10**3
p_valor_est = estimar_p_valor_pearson(t, Nsim, p, k, n)

data = [[p_valor, p_valor_est]]
headers = ["P-valor Pearson", "P-valor simulacion"]
print(tabulate(data, headers=headers, tablefmt="pretty"))

+---------------------+--------------------+
|   P-valor Pearson   | P-valor simulacion |
+---------------------+--------------------+
| 0.36605389988682624 |       0.374        |
+---------------------+--------------------+


In [5]:
## Ejercicio 13
def expo(x):
    if x < 0:
        return 0
    else:
        return 1 - exp(-1 * x)

muestra = generar_muestra(lambda: generar_exp(1), 10)
d = calcular_D(muestra, expo)
n = len(muestra)

# Simulacion
Nsim = 10**4
p_valor = estimar_p_valor_k(d, Nsim, n)

data = [[p_valor, d]]
headers = ["P-valor estimado", "Estadistico"]

print(tabulate(data, headers=headers, tablefmt="pretty"))

+------------------+--------------------+
| p-valor estimado |    Estadistico     |
+------------------+--------------------+
|      0.7183      | 0.2058578067894975 |
+------------------+--------------------+


In [7]:
## Ejericio 14
def rt(df):
    """
    Genera variables aleatorias de una distribucion t-student
    Args:
        df: Grados de libertad de la distribucion
    """
    x = gauss(0.0, 1.0)
    y = 2.0 * gammavariate(0.5*df, 2.0)
    return x / (sqrt(y/df))

def acumulada_z(x):
    """
    Distribucion acumulada de la variable aleatoria normal estandar N(0, 1)
    """
    return erf(x/sqrt(2.))/2.+0.5

# Generamos la muestra de 10, 20, 100 y 1000 elementos de la t_studen de 11 grados
# de libertad
muestra_10 = generar_muestra(lambda: rt(df=11), 10)
muestra_20 = generar_muestra(lambda: rt(df=11), 20)
muestra_100 = generar_muestra(lambda: rt(df=11), 100)
muestra_1000 = generar_muestra(lambda: rt(df=11), 1000)

# Calculo los respectivos estadisticos D
d_10 = calcular_D(muestra_10, acumulada_z)
d_20 = calcular_D(muestra_20, acumulada_z)
d_100 = calcular_D(muestra_100, acumulada_z)
d_1000 = calcular_D(muestra_1000, acumulada_z)

# Estimo los respectivos p-valores
Nsim = 10**3
p_valor_10 = estimar_p_valor_k(d_10, Nsim, 10)
p_valor_20 = estimar_p_valor_k(d_20, Nsim, 20)
p_valor_100 = estimar_p_valor_k(d_100, Nsim, 100)
p_valor_1000 = estimar_p_valor_k(d_1000, Nsim, 1000)

# Tabla de datos
data = [[10, d_10, p_valor_10],
          [20, d_20, p_valor_20],
          [100, d_100, p_valor_100],
          [1000, d_1000, p_valor_1000]]

headers = ["Tamaño de la muestra", "Estadistico", "P-valor estimado"]

print(tabulate(data, headers=headers, tablefmt="pretty"))

+----------------------+---------------------+------------------+
| Tamaño de la muestra |     Estadistico     | P-valor estimado |
+----------------------+---------------------+------------------+
|          10          | 0.2264131953312527  |      0.607       |
|          20          | 0.18817704234015448 |      0.418       |
|         100          | 0.09952729010436051 |      0.242       |
|         1000         | 0.09080320831503397 |       0.0        |
+----------------------+---------------------+------------------+


In [9]:
## Ejercicio 15
def expo(x, Lambda):
    if x < 0:
        return 0
    else:
        return 1 - exp(-Lambda * x)
    
muestra = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3, 10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]
n = len(muestra)

# Estimar parametros desconocidos
Lambda_est = sum(muestra) / n   # Estimamos E[e(λ)] = 1/λ
Lambda_est = 1 / Lambda_est      # Invertimos y obtenemos λ

d = calcular_D(muestra, lambda x: expo(x, Lambda_est))

Nsim = 10**4
p_valor = estimar_p_valor_k(d, Nsim, n)

p_valor_sim = 0
for _ in range(Nsim):
    # Generamos la muestra a partir de la distribucion "original"
    muestra_sim = generar_muestra(lambda: generar_exp(Lambda_est), n)
    # Reestimamos el parametro desconocido
    Lambda_sim = n / sum(muestra_sim)
    # Calculamos el estadistico de la simulacion usando el parametro estimado
    d_sim = calcular_D(muestra_sim, lambda x: expo(x, Lambda_sim))
    if d_sim > d:
        p_valor_sim += 1

p_valor_sim = p_valor_sim / Nsim


data = [[Lambda_est, p_valor, p_valor_sim, d]]
headers = ["Parametro λ estimado",
             "P-valor (uniformes)", "P-valor (no uniformes)", "Estadistico"]


print(tabulate(data, headers=headers, tablefmt="pretty"))

+----------------------+---------------------+------------------------+---------------------+
| Parametro λ estimado | P-valor (uniformes) | P-valor (no uniformes) |     Estadistico     |
+----------------------+---------------------+------------------------+---------------------+
| 0.09322560596643878  |       0.1872        |         0.0517         | 0.26949936321059237 |
+----------------------+---------------------+------------------------+---------------------+


In [11]:
## Ejercicio 16
def acumulada(x, mu, sigma):
    """
    Distribucion acumulada de la variable aleatoria dada como 
    hipotesis nula en el ejercicio 10
    """
    return erf((x-mu)/(sigma*sqrt(2.)))/2.+0.5


def normal_ross(mu=0, sigma=1):
    """
    Genera una variable normal estandar (por defecto) usando el algoritmo descrito 
    en el libro de Sheldon Ross, el cual esta basado en el metodo de aceptacion y 
    rechazo
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    """
    while True:
        Y1 = -log(random())
        Y2 = -log(random())
        if Y2 >= (Y1 - 1) ** 2 / 2:
            if random() < 0.5:
                return Y1 * sigma + mu
            else:
                return -Y1 * sigma + mu
            
muestra = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0, 103.8, 99.6,  96.6, 119.3, 104.8, 101.7]
n = len(muestra)

# Estimar parametros desconocidos

# Estimacion de la media de la normal
mu_est = sum(muestra) / n
# Estimacion de la desviacion
sigma_est = sum(list(map(lambda x: (x - mu_est)**2, muestra))) / (n - 1)
sigma_est = sqrt(sigma_est)

# Calculamos el valor de estimador D
d = calcular_D(muestra, lambda x: acumulada(x, mu_est, sigma_est))

# Estimamos el p-valor
Nsim = 10**4
p_valor = estimar_p_valor_k(d, Nsim, n)

# Estimamos el p-valor con una simulacion mas certera (calculando estadisticos
# de muestras con distribucion H0)
p_valor_sim = 0
for _ in range(Nsim):
    # Generamos la muestra a partir de la distribucion "original"
    muestra_sim = generar_muestra(lambda: normal_ross(mu_est, sigma_est), n)

    # Reestimamos los parametros desconocidos
    mu_sim = sum(muestra_sim) / n
    sigma_sim = sqrt(sum(list(map(lambda x: (x - mu_sim)**2, muestra_sim))) / n)

    # Calculamos el estadistico de la simulacion usando los parametros estimados
    d_sim = calcular_D(muestra_sim, lambda x: acumulada(x, mu_sim, sigma_sim))

    if d_sim > d:
        p_valor_sim += 1

p_valor_sim = p_valor_sim / Nsim


data = [[mu_est, sigma_est, p_valor, p_valor_sim]]
headers = ["μ estiamdo", "σ estimado", "P-valor (uniformes)", "P-valor (no uniformes)"]


print(tabulate(data, headers=headers, tablefmt="pretty"))

+--------------------+-------------------+---------------------+------------------------+
|     μ estiamdo     |    σ estimado     | P-valor (uniformes) | P-valor (no uniformes) |
+--------------------+-------------------+---------------------+------------------------+
| 104.13333333333333 | 9.397420664816233 |       0.6707        |         0.2466         |
+--------------------+-------------------+---------------------+------------------------+
